In [ ]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget import neurons
from voltagebudget.budget import locate_firsts
from voltagebudget.budget import locate_peaks
from voltagebudget.budget import filter_voltages
from fakespikes import util as fsutil

# Define input

In [ ]:
t = 1.0

k = 50
stim_onset = 0.6
stim_offset = 0.6 + 50e-3
stim_rate = 12
dt = 1e-5
seed_stim = 1
ns, ts = util.poisson_impulse(
        t,
        stim_onset,
        stim_offset - stim_onset,
        stim_rate,
        n=k,
        dt=dt,
        seed=None)

print(">>> {} spikes".format(ts.size))
print(">>> {} population rate".format(ts.size / (stim_offset - stim_onset)))

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts, ns, color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, 1)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

- Shadow voltage response of an LIF neuron to the input

In [ ]:
#set mode
mode = 'regular'
params, w_max, bias, sigma = util.read_modes(mode)

In [ ]:
print(w_max, bias, sigma)

In [ ]:
# -
v_in, b_in = neurons.shadow_adex(10, t, ns, ts, w_max=w_max, bias=bias, sigma=sigma, **params)
times = b_in['times']

# -
p = figure(plot_width=400, plot_height=200)
for n in range(10):
    p.line(x=times, y=v_in[n, :], color="black", alpha=0.1)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
combine = False
ns, ts = locate_peaks(b_in, stim_onset, stim_offset, combine=combine)
# -
p = figure(plot_width=400, plot_height=200)
for n in range(10):
    p.line(x=times, y=v_in[n, :], color="black", alpha=0.1)
    if not combine:
        p.circle(x=ts[n], y=v_in[n, :].max(), color="purple")
    else:
        p.line(x=ts[0], y=np.linspace(v_in[:, :].min(), v_in[:, :].max(), 10), color="purple")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Define computational models

And obseve how they respond to the same input

## Regular

In [ ]:
N = 50
ns_y, ts_y, budget = neurons.adex(N, t, ns, ts, 
                                  w_max=w_max, 
                                  bias=bias, 
                                  sigma=sigma, 
                                  A=.0, f=0, **params)

In [ ]:
print(ns_y, ts_y)

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_y, ns_y, color="black")
p.x_range = Range1d(0.0, t)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

print(ts_y.size)

In [ ]:
times = budget['times']
m = times > 0.05  # Burn in time

p = figure(plot_width=400, plot_height=200)
for n in range(20):
    v = budget['V_m'][n, :]
    p.line(x=times[m], y=v[m], color="black", alpha=0.1)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
# Extract budgets
combine = False
ns_first, ts_first = locate_firsts(ns_y, ts_y, combine=combine)
voltages_m = filter_voltages(
    budget,
    ns_first,
    ts_first,
    budget_delay=-10e-3,
    budget_width=2e-3,
    combine=combine)

times = budget['times']


p = figure(plot_width=400, plot_height=200)

for n in range(N):
    v = voltages_m['V_m'][n, :]
    if not combine:
        times = voltages_m['times'][n, :]
    else:
        times = voltages_m['times']
    
    p.line(x=times, y=v, color="purple", alpha=0.8)
    
    v_full = budget['V_m'][n, :]
    times_full = budget['times']
    p.line(x=times_full, y=v_full, color="black", alpha=0.1)
    
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.x_range = Range1d(0.63, 0.7)
p.y_range = Range1d(-66e-3, -30e-3)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
print(voltages_m['V_m'].shape, voltages_m['times'].shape)

In [ ]:
times = budget['times']
v = budget['V_comp'][0, :]
m = times > 0.05  # Burn in time

p = figure(plot_width=400, plot_height=200)
p.line(x=times[m], y=v[m], color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
times = budget['times']
v = budget['V_osc'][0, :]
m = times > 0.05  # Burn in time

p = figure(plot_width=400, plot_height=200)
p.line(x=times[m], y=v[m], color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
times = budget['times']
v = budget['V_free'][0, :]
m = times > 0.05  # Burn in time

p = figure(plot_width=400, plot_height=200)
p.line(x=times[m], y=v[m], color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)